### Importando as Bibliotecas e Funções

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def drop_reset_index(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.index += 1
    return df

def plot_profit_acu(dataframe, title_text):
    dataframe['Profit_acu'] = dataframe.Profit.cumsum()
    dataframe['Investimento'] = 1
    n_apostas = dataframe.shape[0]
    profit = round(dataframe.Profit_acu.tail(1).item(), 2)
    dataframe['Investimento_acu'] = dataframe.Investimento.cumsum()
    ROI = round(((dataframe.Profit_acu.tail(1) / dataframe.Investimento_acu.tail(1)) * 100).item(), 2)
    drawdown = dataframe['Profit_acu'] - dataframe['Profit_acu'].cummax()
    drawdown_maximo = round(drawdown.min(), 2)
    winrate_medio = round((dataframe['Profit'] > 0).mean() * 100, 2)
    desvio_padrao = round(dataframe['Profit'].std(), 2)

    ax = dataframe.Profit_acu.plot(title=title_text, xlabel='Entradas', ylabel='Stakes')
    ax.set_title(title_text)
    ax.set_xlabel('Entradas')
    ax.set_ylabel('Stakes')

    print("Metodo:", title_text)
    print("Profit:", profit, "stakes em", n_apostas, "jogos")
    print("ROI:", ROI, "%")
    print("Drawdown Maximo Acumulado:", drawdown_maximo)
    print("Winrate Medio:", winrate_medio, "%")
    print("Desvio Padrao:", desvio_padrao)
    print("")

    plt.show()

### Importando a Base de Dados

In [ ]:
df = pd.read_excel('https://github.com/futpythontrader/YouTube/raw/refs/heads/main/Bases_de_Dados/FootyStats/Base_de_Dados_FootyStats_(2022_2024).xlsx')
df = df[['League','Season','Date','Rodada','Home','Away',
         'Goals_H_FT','Goals_A_FT','TotalGoals_FT','Goals_H_Minutes','Goals_A_Minutes',
         'Odd_H_FT','Odd_D_FT','Odd_A_FT','Odd_Over25_FT','Odd_Under25_FT','Odd_BTTS_Yes','Odd_BTTS_No','Odd_DC_1X','Odd_DC_12','Odd_DC_X2']]
df = drop_reset_index(df)
display(df)

In [ ]:
columns_to_filter = [
    'Odd_Over25_FT', 'Odd_Under25_FT', 'Odd_BTTS_Yes',
    'Odd_BTTS_No', 'Odd_DC_1X', 'Odd_DC_12', 'Odd_DC_X2'
]
df = df[(df[columns_to_filter] != 0).all(axis=1)]
df = drop_reset_index(df)

display(df)

### Profits dos Mercados de MO, OU25 e BTTS

In [ ]:
df['Back_Home'] = np.where((df['Goals_H_FT'] > df['Goals_A_FT']), 1, 0)
df['Profit_Back_Home'] = np.where((df['Back_Home'] == 1), df['Odd_H_FT']-1, -1)

df['Back_Draw'] = np.where((df['Goals_H_FT'] == df['Goals_A_FT']), 1, 0)
df['Profit_Back_Draw'] = np.where((df['Back_Draw'] == 1), df['Odd_D_FT']-1, -1)

df['Back_Away'] = np.where((df['Goals_H_FT'] < df['Goals_A_FT']), 1, 0)
df['Profit_Back_Away'] = np.where((df['Back_Away'] == 1), df['Odd_A_FT']-1, -1)

display(df)

In [ ]:
df['Lay_Home'] = np.where((df['Goals_H_FT'] <= df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Home'] = np.where((df['Lay_Home'] == 1), df['Odd_DC_X2']-1, -1)

df['Lay_Draw'] = np.where((df['Goals_H_FT'] != df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Draw'] = np.where((df['Lay_Draw'] == 1), df['Odd_DC_12']-1, -1)

df['Lay_Away'] = np.where((df['Goals_H_FT'] >= df['Goals_A_FT']), 1, 0)
df['Profit_Lay_Away'] = np.where((df['Lay_Away'] == 1), df['Odd_DC_1X']-1, -1)

display(df)

In [ ]:
df['Over_25'] = np.where((df['TotalGoals_FT'] > 2), 1, 0)
df['Profit_Over_25'] = np.where((df['Over_25'] == 1), df['Odd_Over25_FT']-1, -1)

df['Under_25'] = np.where((df['TotalGoals_FT'] < 3), 1, 0)
df['Profit_Under_25'] = np.where((df['Under_25'] == 1), df['Odd_Under25_FT']-1, -1)

display(df)

In [ ]:
df['BTTS_Yes'] = np.where(((df['Goals_H_FT'] > 0) & (df['Goals_A_FT'] > 0)), 1, 0)
df['Profit_BTTS_Yes'] = np.where((df['BTTS_Yes'] == 1), df['Odd_BTTS_Yes']-1, -1)

df['BTTS_No'] = np.where(((df['Goals_H_FT'] == 0) | (df['Goals_A_FT'] == 0)), 1, 0)
df['Profit_BTTS_No'] = np.where((df['BTTS_No'] == 1), df['Odd_BTTS_No']-1, -1)

display(df)

In [ ]:
modelo = 'Back_Home'

df['Profit'] = df[f'Profit_{modelo}']
plot_profit_acu(df, f'{modelo}')

### Probabilidades, CV MO, Variáveis Derivadas das Odds

### Variáveis de Desempenho

### Variáveis de Desempenho Ponderadas Pela Odd